In [ ]:
import numpy as np
import re
import subprocess
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
from astropy.io import fits
from synphot import SpectralElement
from synphot.models import Empirical1D

import os
gd_folder = os.getcwd()+"/../../Calculations/SKIRTOR/"

import sys 
sys.path.append(gd_folder)
#from loadSKIRTOR_MRN77 import LoadSKIRTOR_MRN77
#from loadSKIRTOR_MRN77_old import LoadSKIRTOR_MRN77
from loadSKIRTOR_MRN77_force_reg import LoadSKIRTOR_MRN77
from loadSKIRTOR_General import LoadSKIRTOR_General

sys.path.append("../utility_functions/")
from objectProperties import ObjectProperties
from readExtrapolatedSpectra import ReadExtrapolatedSpectra

sys.path.append("../../Filter_Curves/")
from readBands import ReadBands

In [ ]:
#Load the properties of the BHDs
op = ObjectProperties(wids=["W0204-0506"])

In [ ]:
#Load the bands.
bands = ReadBands()
for bname in bands.bandnames:
    print(bname, bands.bp[bname].barlam(), bands.bp[bname].fwhm(), bands.bp[bname].rectwidth())

In [ ]:
#Load the spectra for each BHD, extended using the best-fit SED.
specs = ReadExtrapolatedSpectra()

In [ ]:
def get_p_chi2_map_bb(p_meas, p_unc, bands, specs, wid, pw):

    #Set the grid of values for the calculation. 
    tang_grid = np.arange(25., 60.+0.01, 0.5)*u.deg
    cang_grid = np.arange(10., 55.+0.01, 0.5)*u.deg
    iang_grid = np.arange(25., 90.+0.01, 0.5)*u.deg
    # tang_grid = np.arange(25., 60.+0.01, 0.5)*u.deg
    # cang_grid = np.arange(20., 55.+0.01, 0.5)*u.deg
    # iang_grid = np.arange(25., 90.+0.01, 0.5)*u.deg
    #tang_grid = np.arange(20., 70., 2.5)*u.deg
    #cang_grid = np.arange(10., 60., 2.5)*u.deg
    #iang_grid = np.arange(20., 85., 2.5)*u.deg
    # tang_grid = np.arange(20., 70., 5)*u.deg
    # cang_grid = np.arange(10., 60., 5)*u.deg
    # iang_grid = np.arange(20., 85., 5)*u.deg


    #Create the output array. We'll mask certain regions that are not technically allowed by the model. 
    p_map = np.ma.zeros((len(bands), len(tang_grid), len(cang_grid), len(iang_grid)))
    p_map.mask = np.zeros(p_map.shape, dtype=bool)

    for j, band in enumerate(bands):
        #p_map[j,:,:,:] = pw.p_bb(band, tang_grid, cang_grid, iang_grid, specs.lam_obs[wid], specs.flam[wid], specs.specs.sp[wid].zspec)
        p_map[j,:,:,:] = pw.p_bb(band, tang_grid, cang_grid, iang_grid, specs.lam_obs[wid], specs.flam[wid], specs.specs.sp[wid].zspec)

    #Turn into percentages. 
    p_map *= 100
    
    #Remove angles were we would directly see the accretion disk. 
    #p_map[th_2d_grid<psi_2d_grid] = np.nan
    #p_map.mask[th_2d_grid<psi_2d_grid] = True

    #Transform it into a chi-squared map
    chi2_map = np.ma.zeros(p_map.shape[1:])
    chi2_map.mask = np.zeros(chi2_map.shape, dtype=bool)
    for j in range(len(bands)):
        chi2_map += ((p_map[j]-p_meas[j])/p_unc[j])**2

    return p_map, chi2_map, tang_grid, cang_grid, iang_grid


In [ ]:
def save_p_chi2(chi2_map, tang_grid, cang_grid, iang_grid, wid, folder="maps_BottomCone"):

    if not os.path.exists(folder):
        subprocess.call(["mkdir",folder])
    
    fname = "{}/chi2_map_{}.fits".format(folder, wid)

    chi2_hdr = fits.Header()

    chi2_hdr['TANGGRID'] = " ".join(tang_grid.value.astype(str))
    chi2_hdr['TANGUNIT'] = tang_grid.unit._short_names[0]
    chi2_hdr['CANGGRID'] = " ".join(cang_grid.value.astype(str))
    chi2_hdr['CANGUNIT'] = cang_grid.unit._short_names[0]
    chi2_hdr['IANGGRID'] = " ".join(iang_grid.value.astype(str))
    chi2_hdr['IANGUNIT'] = iang_grid.unit._short_names[0]

    chi2_data_hdu = fits.PrimaryHDU(data=chi2_map.data)
    chi2_data_hdu.header.update(chi2_hdr)
    chi2_mask_hdu = fits.ImageHDU(data=chi2_map.mask.astype(int))

    hdul = fits.HDUList([chi2_data_hdu, chi2_mask_hdu])
    hdul.writeto(fname, overwrite=True)

    return

In [ ]:
#Calculate the chi2 maps for each model. 
force_new=False
#force_new=True
p_map = dict()
chi2_map = dict()
pw = LoadSKIRTOR_MRN77(cone_type='Bottom', interp_method='pchip')
for wid in op.wids:
    bands_use = list()
    p_meas = list()
    p_unc = list()
    for band in op.pfrac[wid].keys():
        bands_use.append(bands.bp[band])
        p_meas.append(op.pfrac[wid][band])
        p_unc.append(op.epfrac[wid][band])

    fname = "maps_BottomCone/chi2_map_{}.fits".format(wid)
    if os.path.exists(fname) and not force_new:
        h = fits.open(fname)
        tang_grid = np.array(h[0].header['TANGGRID'].split()).astype(float)
        tang_grid = tang_grid * u.Unit(h[0].header['TANGUNIT'])
        cang_grid = np.array(h[0].header['CANGGRID'].split()).astype(float)
        cang_grid = cang_grid * u.Unit(h[0].header['CANGUNIT'])
        iang_grid = np.array(h[0].header['IANGGRID'].split()).astype(float)
        iang_grid = iang_grid * u.Unit(h[0].header['IANGUNIT'])
        chi2_map[wid] = np.ma.zeros((len(tang_grid), len(cang_grid), len(iang_grid)))
        chi2_map[wid][:,:,:] = h[0].data
        chi2_map[wid].mask = h[1].data.astype(bool)
        print(wid, chi2_map[wid][0,0,0])
    else:
        p_map[wid], chi2_map[wid], tang_grid, cang_grid, iang_grid = get_p_chi2_map_bb(p_meas, p_unc, bands_use, specs, wid, pw)
        print(wid, chi2_map[wid][0,0,0])
        save_p_chi2(chi2_map[wid], tang_grid, cang_grid, iang_grid, wid)

In [ ]:
#Find the best solution for W0116-0505
i, j, k = np.unravel_index(np.argmin(chi2_map['W0116-0505'], axis=None), chi2_map['W0116-0505'].shape)
print(tang_grid[i], cang_grid[j], iang_grid[k], chi2_map['W0116-0505'].min())

In [ ]:
#Find the uncertainty as well. 
wid = 'W0116-0505'
#indices = np.argwhere(chi2_map[wid]<=np.min(chi2_map[wid])+1.0)
indices = np.argwhere(chi2_map[wid]<=2.0 * np.min(chi2_map[wid]))
print(np.min(tang_grid[indices[:,0]]), np.max(tang_grid[indices[:,0]]))
print(np.min(cang_grid[indices[:,1]]), np.max(cang_grid[indices[:,1]]))
print(np.min(iang_grid[indices[:,2]]), np.max(iang_grid[indices[:,2]]))

In [ ]:
lam_grid = np.arange(1200., 3000., 20.)*u.AA

#Plot the range of models. 
#indices = np.argwhere(chi2_map[wid]<=np.min(chi2_map[wid])+1.0)
indices = np.argwhere(chi2_map[wid]<=np.min(2.0*chi2_map[wid]))
curve_min = np.zeros(len(lam_grid)) + 1e32
curve_max = np.zeros(curve_min.shape)
for ind in indices:
    i, j, k = ind
    paux = 100*pw.p((tang_grid[i]*np.ones(lam_grid.shape), cang_grid[j]*np.ones(lam_grid.shape), iang_grid[k]*np.ones(lam_grid.shape), lam_grid.value))
    curve_min = np.where(paux < curve_min, paux, curve_min)
    curve_max = np.where(paux > curve_max, paux, curve_max)
plt.fill_between(lam_grid.value, curve_min, curve_max, color='cyan', alpha=0.5)

#Now plot the best fit. 
i, j, k = np.unravel_index(np.argmin(chi2_map[wid], axis=None), chi2_map[wid].shape)
plt.plot(lam_grid, 100*pw.p((tang_grid[i]*np.ones(lam_grid.shape), cang_grid[j]*np.ones(lam_grid.shape), iang_grid[k]*np.ones(lam_grid.shape), lam_grid)))

wid = "W0116-0505"
for bname in op.pfrac[wid].keys():
    plt.errorbar([bands.bp[bname].barlam().value/(1+op.z[wid])], [op.pfrac[wid][bname]], xerr=[bands.bp[bname].rectwidth().value/2./(1+op.z[wid])], yerr=[op.epfrac[wid][bname]], marker='o', color='k')

    #p_mod = pw.p_bb(bands.bp[bname], [tang_grid[i]], [cang_grid[j]], [iang_grid[k]], specs.lam_obs[wid], specs.flam[wid], op.z[wid])
    p_mod = pw.p_bb(bands.bp[bname], tang_grid[i:i+1], cang_grid[j:j+1], iang_grid[k:k+1], specs.lam_obs[wid], specs.flam[wid], op.z[wid])
    plt.plot([bands.bp[bname].barlam().value/(1+op.z[wid])], [p_mod[0,0,0]*100], 'bx')
    

In [ ]:
#Obtained the combined chi2 of the fit. Specifically, we optimize the fit for every objects for a fixed combination of torus and cone openings and generate a chi2 map in this way. 
chi2_all = np.zeros(chi2_map[op.wids[0]].shape[:2])
for wid in op.wids:
    chi2_all += np.min(chi2_map[wid], axis=2)

In [ ]:
i, j = np.unravel_index(np.argmin(chi2_all, axis=None), chi2_all.shape)
print(tang_grid[i], cang_grid[j], chi2_all.min())
for wid in op.wids:
    print(wid, iang_grid[np.argmin(chi2_map[wid][i,j])], np.min(chi2_map[wid][i,j]))

In [ ]:
chi2_all_masked = np.zeros(chi2_map[op.wids[0]].shape[:2])
for wid in op.wids:
    for i, tang in enumerate(tang_grid):
        for j, cang in enumerate(cang_grid):
            if tang>cang:
                cond = iang_grid>tang
                chi2_all_masked[i,j] += np.min(chi2_map[wid][i,j,cond])
            else:
                chi2_all_masked[i,j] = np.inf

In [ ]:
i, j = np.unravel_index(np.argmin(chi2_all_masked, axis=None), chi2_all.shape)
print(tang_grid[i], cang_grid[j], chi2_all_masked.min())
for wid in op.wids:
    print(wid, iang_grid[np.argmin(chi2_map[wid][i,j])], np.min(chi2_map[wid][i,j]))

In [ ]:
#Set the additional SKIRTOR models we want to plot for W0116-0505. 
wid = "W0116-0505"
other_sk_models = [
    {"d03_mod":"MW"},
    {"d03_mod":"LMC"},
    {"d03_mod":"SMC"},
    #{"gs_min":0.005, "gs_max":0.25, "a":3.5},
    {"gs_min":1, "gs_max":10, "a":3.5, "sil":True},
    #{"gs_min":0.1, "gs_max":1, "a":0.0},
    #{"gs_min":0.1, "gs_max":1, "a":3.5},
    {"gs_min":0.1, "gs_max":1, "a":3.5, "sil":True},
]
other_sk = list()
other_chi2 = list()
for oskm in other_sk_models:
    other_sk.append(LoadSKIRTOR_General(**oskm))
    chi2_aux = np.zeros(len(iang_grid))
    for bname in op.pfrac[wid].keys():
        p_aux = other_sk[-1].p_bb(bands.bp[bname], iang_grid, specs.lam_obs[wid], specs.flam[wid], specs.specs.sp[wid].zspec)
        chi2_aux += ((p_aux*100 - op.pfrac[wid][bname])/op.epfrac[wid][bname])**2
    other_chi2.append(chi2_aux)


In [ ]:
i1, j1, k1 = np.unravel_index(np.argmin(chi2_map[wid], axis=None), chi2_map[wid].shape)
wid, tang_grid[i1], cang_grid[j1], iang_grid[k1]

In [ ]:
wid = "W0116-0505"
#indices = np.argwhere(chi2_map[wid]<1.0+np.min(chi2_map[wid]))
indices = np.argwhere(chi2_map[wid]<2.0*np.min(chi2_map[wid]))
index_min = np.unravel_index(np.argmin(chi2_map[wid], axis=None), chi2_map[wid].shape)
print(np.min(tang_grid[indices[:,0]]), np.max(tang_grid[indices[:,0]]), tang_grid[index_min[0]])
print(np.min(cang_grid[indices[:,1]]), np.max(cang_grid[indices[:,1]]), cang_grid[index_min[1]])
print(np.min(iang_grid[indices[:,2]]), np.max(iang_grid[indices[:,2]]), iang_grid[index_min[2]])
print(np.min(chi2_map[wid]))

In [ ]:
#indices = np.argwhere(chi2_all_masked<1.0+np.min(chi2_all_masked))
indices = np.argwhere(chi2_all_masked<2.0*np.min(chi2_all_masked))
index_min = np.unravel_index(np.argmin(chi2_all_masked, axis=None), chi2_all_masked.shape)
print(np.min(tang_grid[indices[:,0]]), np.max(tang_grid[indices[:,0]]), tang_grid[index_min[0]])
print(np.min(cang_grid[indices[:,1]]), np.max(cang_grid[indices[:,1]]), cang_grid[index_min[1]])
print(np.min(chi2_all))

In [ ]:
indices = np.argwhere(chi2_all_masked<1.0+np.min(chi2_all_masked))
index_min = np.unravel_index(np.argmin(chi2_all_masked, axis=None), chi2_all_masked.shape)
print(np.min(tang_grid[indices[:,0]]), np.max(tang_grid[indices[:,0]]), tang_grid[index_min[0]])
print(np.min(cang_grid[indices[:,1]]), np.max(cang_grid[indices[:,1]]), cang_grid[index_min[1]])
print(np.min(chi2_all))

In [ ]:
indices = np.argwhere(chi2_all<1.0+np.min(chi2_all))
print(np.min(tang_grid[indices[:,0]]), np.max(tang_grid[indices[:,0]]))
print(np.min(cang_grid[indices[:,1]]), np.max(cang_grid[indices[:,1]]))

In [ ]:
colors = {"W0116-0505":"xkcd:green", "W0019-1046":"magenta", "W0204-0506":"blue", "W0220+0137":"red", "W0831+0140":"cyan"}

fig, axs = plt.subplots(1,2, figsize=(10,4), sharey=True, sharex=True)
plt.subplots_adjust(wspace=0., hspace=0.)

for ax in axs:
    ax.tick_params(axis="both", direction="in", labelsize=12, bottom=True, top=True, left=True, right=True)

#On one plot, we show the best solution for W0116-0505. 
lam_grid = np.arange(1200., 3000., 20.)*u.AA
wid = "W0116-0505"

#First plot the range of plaussible models. 
#indices = np.argwhere(chi2_map[wid]<=np.min(chi2_map[wid])+1.0)
indices = np.argwhere(chi2_map[wid]<=2.0*np.min(chi2_map[wid]))
curve_min = np.zeros(len(lam_grid)) + 1e32
curve_max = np.zeros(curve_min.shape)
for ind in indices:
    i, j, k = ind
    paux = 100*pw.p((tang_grid[i]*np.ones(lam_grid.shape), cang_grid[j]*np.ones(lam_grid.shape), iang_grid[k]*np.ones(lam_grid.shape), lam_grid.value))
    curve_min = np.where(paux < curve_min, paux, curve_min)
    curve_max = np.where(paux > curve_max, paux, curve_max)
axs[0].fill_between(lam_grid.value, curve_min, curve_max, color=colors[wid], alpha=0.2)

#labels = ['Draine (2003) Mixtures', None, None, 'Small grains graphite only', 'Large grains', 'MRN77 + silicate']
labels = ['Draine (2003) Mixtures', None, None, 'Large grains', 'MRN77 + silicate']
for mm, osk in enumerate(other_sk):
    kk = np.argmin(other_chi2[mm])
    ls = 'dashed'
    if mm==3:
        ls = 'dashdot'
    elif mm==4:
        ls = (5, (10,3))
    # if mm>2 and mm<5:
    #     ls = 'dashdot'
    elif mm==5:
        ls = 'dotted'
    axs[0].plot(lam_grid, 100*osk.p((iang_grid[kk]*np.ones(lam_grid.shape), lam_grid)), c='xkcd:grey',ls=ls, label=labels[mm])
i1, j1, k1 = np.unravel_index(np.argmin(chi2_map[wid], axis=None), chi2_map[wid].shape)
axs[0].plot(lam_grid, 100*pw.p((tang_grid[i1]*np.ones(lam_grid.shape), cang_grid[j1]*np.ones(lam_grid.shape), iang_grid[k1]*np.ones(lam_grid.shape), lam_grid)), c=colors[wid],ls='solid', label='MRN77')

for bname in op.pfrac[wid].keys():

    axs[0].errorbar([bands.bp[bname].barlam().value/(1+op.z[wid])], [op.pfrac[wid][bname]], xerr=[bands.bp[bname].rectwidth().value/2./(1+op.z[wid])], yerr=[op.epfrac[wid][bname]], marker='o', c=colors[wid], markeredgecolor='black')

    #p_mod = pw.p_bb(bands.bp[bname], [tang_grid[i1]], [cang_grid[j1]], [iang_grid[k1]], specs.lam_obs[wid], specs.flam[wid], op.z[wid])
    p_mod = pw.p_bb(bands.bp[bname], tang_grid[i1:i1+1], cang_grid[j1:j1+1], iang_grid[k1:k1+1], specs.lam_obs[wid], specs.flam[wid], op.z[wid])
    axs[0].plot([bands.bp[bname].barlam().value/(1+op.z[wid])], [p_mod[0,0,0]*100], c=colors[wid], marker='x')

axs[0].text(0.05, 0.9, "W0116-0505", transform=axs[0].transAxes, fontsize=16, weight='bold')
axs[0].text(0.05, 0.82, r"$\psi_{{\rm Torus}} = {:.0f}^{{\circ}}$".format(tang_grid[i1].value), transform=axs[0].transAxes, fontsize=14)
axs[0].text(0.05, 0.74, r"$\psi_{{\rm Cone}}  = {:.0f}^{{\circ}}$".format(cang_grid[j1].value), transform=axs[0].transAxes, fontsize=14)
axs[0].text(0.05, 0.66, r"$\eta = {:.0f}^{{\circ}}$".format(iang_grid[k1].value), transform=axs[0].transAxes, fontsize=14)

axs[0].legend()#loc=(0.45, 0.55))

#On the other, we will show the combined best fit.  
lam_grid = np.arange(1200., 3000., 20.)*u.AA
for wid in op.wids:
    i2, j2 = np.unravel_index(np.argmin(chi2_all_masked, axis=None), chi2_all.shape)
    for bname in op.pfrac[wid].keys():

        k2 = np.argmin(chi2_map[wid][i2,j2,:])

        if bname == "R_SPECIAL":
            label = r"{} $\eta$=${}^{{\circ}}$".format(re.sub("-","$-$",wid), iang_grid[k2].value)
        else:
            label = None

        axs[1].plot(lam_grid, 100*pw.p((tang_grid[i2]*np.ones(lam_grid.shape), cang_grid[j2]*np.ones(lam_grid.shape), iang_grid[k2]*np.ones(lam_grid.shape), lam_grid)), c=colors[wid],ls='solid')

        axs[1].errorbar([bands.bp[bname].barlam().value/(1+op.z[wid])], [op.pfrac[wid][bname]], xerr=[bands.bp[bname].rectwidth().value/2./(1+op.z[wid])], yerr=[op.epfrac[wid][bname]], marker='o', color=colors[wid], markeredgecolor='black', label=label)

        #p_mod = pw.p_bb(bands.bp[bname], [tang_grid[i2]], [cang_grid[j2]], [iang_grid[k2]], specs.lam_obs[wid], specs.flam[wid], op.z[wid])
        p_mod = pw.p_bb(bands.bp[bname], tang_grid[i2:i2+1], cang_grid[j2:j2+1], iang_grid[k2:k2+1], specs.lam_obs[wid], specs.flam[wid], op.z[wid])
        axs[1].plot([bands.bp[bname].barlam().value/(1+op.z[wid])], [p_mod[0,0,0]*100], c=colors[wid], marker='x')

axs[1].text(0.05, 0.9, "Combined Best-fit", transform=axs[1].transAxes, fontsize=16, weight='bold')
axs[1].text(0.05, 0.82, r"$\psi_{{\rm Torus}} = {:.0f}^{{\circ}}$".format(tang_grid[i2].value), transform=axs[1].transAxes, fontsize=14)
axs[1].text(0.05, 0.74, r"$\psi_{{\rm Cone}}  = {:.0f}^{{\circ}}$".format(cang_grid[j2].value), transform=axs[1].transAxes, fontsize=14)
#axs[1].text(0.05, 0.66, r"$\eta = {:.0f}^{{\circ}}$".format(iang_grid[k1].value), transform=axs[1].transAxes, fontsize=14)
axs[1].legend()

axs[0].set_ylim([0., 45.])

axs[0].set_ylabel(r"$P (\%)$", fontsize=16)
fig.text(0.5, 0., r"Rest-frame Wavelength $(\rm \AA)$", ha='center', fontsize=16)

fig.savefig("SKIRTOR_MRN77_best-fit.png", dpi=100, bbox_inches='tight')


In [ ]:
wid = "W0116-0505"

fig, axs = plt.subplots(1, 3, figsize=(12,6), layout='constrained')
for ax in axs.flatten():
   #ax.tick_params(axis='both', which='major', labelsize=14)
   #ax.tick_params(axis='both', which='minor', labelsize=12)
   ax.tick_params(axis='both', direction='in', labelsize=14, bottom=True, top=True, left=True, right=True)
#    ax.set_aspect('auto')

chi2_use = np.ma.zeros(chi2_map[wid].shape)
chi2_use.mask = np.zeros(chi2_use.shape, dtype=bool)
chi2_use[:,:,:] = np.copy(chi2_map[wid])
chi2_use /= np.min(chi2_map[wid])

vmin = 0. #np.min(chi2_use)
vmax = 3.0
cmap = 'YlGnBu_r'
#cmap = 'gray'

axis_label_fontsize = 14

II, JJ, KK = np.meshgrid(tang_grid.value, cang_grid.value, iang_grid.value, indexing='ij')
chi2_use.mask[(JJ+5>II) | (KK<II)] = True
#chi2_use.mask[(JJ+10>II) | (II>KK)] = True
#chi2_use.mask[(JJ+15>II) | (II>KK)] = True
chi2_use.mask[chi2_use>vmax] = True

#Torus - Cone
ax = axs[0]
chi2_aux = np.min(chi2_use, axis=2)
cb = ax.imshow(chi2_aux, origin='lower', vmin=vmin, vmax=vmax, cmap=cmap, extent=[np.min(cang_grid.value), np.max(cang_grid.value), np.min(tang_grid.value), np.max(tang_grid.value)])
ax.set_xlim(ax.get_xlim())
ax.set_ylim(ax.get_ylim())
ax.plot(ax.get_xlim(), np.array(ax.get_xlim())+5, 'k--') 
ax.fill_between(ax.get_xlim(), np.array(ax.get_xlim())+5, ax.get_ylim()[0], color="none", hatch="xx", edgecolor="xkcd:gray", linewidth=0.0, alpha=0.7)
ax.set_xlabel(r"Cone half-opening angle $\psi_{\rm Cone}$ (deg)", fontsize=axis_label_fontsize)
ax.set_ylabel(r"Torus half-opening angle $\psi_{\rm Torus}$ (deg)", fontsize=axis_label_fontsize)
x0,x1 = ax.get_xlim()
y0,y1 = ax.get_ylim()
ax.set_aspect(abs(x1-x0)/abs(y1-y0))
ax.text(33, 35, "Cone Illumination Limit", rotation=np.arctan2(abs(x1-x0), abs(y1-y0))*180/np.pi, fontsize=12, weight='extra bold', fontfamily='sans-serif')

#Torus - Inclination
ax = axs[1]
chi2_aux = np.min(chi2_use, axis=1)
ax.imshow(chi2_aux, origin='lower', vmin=vmin, vmax=vmax, cmap=cmap, extent=[np.min(iang_grid.value), np.max(iang_grid.value), np.min(tang_grid.value), np.max(tang_grid.value)])
ax.set_xlim(ax.get_xlim())
ax.set_ylim(ax.get_ylim())
ax.plot(ax.get_xlim(), ax.get_xlim(), 'k--') 
ax.fill_between(ax.get_xlim(), np.array(ax.get_xlim()), ax.get_ylim()[1], color="none", hatch="xx", edgecolor="xkcd:gray", linewidth=0.0, alpha=0.7)
ax.set_xlabel(r"Inclination angle $\eta$ (deg)", fontsize=axis_label_fontsize)
ax.set_ylabel(r"Torus half-opening angle $\psi_{\rm Torus}$ (deg)", fontsize=axis_label_fontsize)
x0,x1 = ax.get_xlim()
y0,y1 = ax.get_ylim()
ax.set_aspect(abs(x1-x0)/abs(y1-y0))
ax.text(28, 33, "Direct line of sight to accretion disk", rotation=np.arctan2(abs(x1-x0), abs(y1-y0))*180/np.pi, fontsize=12, weight='extra bold', fontfamily='sans-serif')

#Cone - Inclination
ax = axs[2]
chi2_aux = np.min(chi2_use, axis=0)
cb = plt.imshow(chi2_aux, origin='lower', vmin=vmin, vmax=vmax, cmap=cmap, extent=[np.min(iang_grid.value), np.max(iang_grid.value), np.min(cang_grid.value), np.max(cang_grid.value)])
ax.set_xlim(ax.get_xlim())
ax.set_ylim(ax.get_ylim())
ax.plot(ax.get_xlim(), np.array(ax.get_xlim())-5, 'k--') 
ax.fill_between(ax.get_xlim(), np.array(ax.get_xlim())-5, ax.get_ylim()[1], color="none", hatch="xx", edgecolor="xkcd:gray", linewidth=0.0, alpha=0.7)
plt.xlabel(r"Inclination angle $\eta$ (deg)", fontsize=axis_label_fontsize)
plt.ylabel(r"Cone half-opening angle $\psi_{\rm Cone}$ (deg)", fontsize=axis_label_fontsize)
x0,x1 = ax.get_xlim()
y0,y1 = ax.get_ylim()
ax.set_aspect(abs(x1-x0)/abs(y1-y0))
ax.text(27, 27, "Direct line of sight to accretion disk", rotation=np.arctan2(abs(x1-x0), abs(y1-y0))*180/np.pi, fontsize=12, weight='extra bold', fontfamily='sans-serif')

fig.colorbar(cb, fraction=0.05).set_label(label=r'Min $\chi^2$',size=18)

plt.show()
fig.savefig("W0116_chi2maps.png", dpi=200, bbox_inches='tight')